# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

from pprint import pprint



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data = pd.read_csv("../output_data/cities.csv")
data

,lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,country,date
0,-53.15,-70.92,272.15,92,18,5.10,CL,1594463357
1,1.48,103.39,300.93,94,20,1.00,MY,1594463530
2,-9.88,-56.09,292.60,65,0,1.25,BR,1594463530
3,54.48,53.47,297.31,79,95,5.09,RU,1594463530
4,-33.65,115.33,284.26,58,1,4.15,AU,1594463530
...,...,...,...,...,...,...,...,...
561,-5.13,-49.33,296.15,94,47,0.89,BR,1594463610
562,-0.72,8.78,300.15,78,40,5.80,GA,1594463641
563,37.78,-121.98,292.59,51,1,1.23,US,1594463392
564,35.51,27.21,301.15,61,20,14.40,GR,1594463641


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [27]:
#set location and humidity
locations = data[["lat","lng"]]
hum = data["Humidity"].astype(float)

#plot heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = hum, dissipating = False, max_intensity = hum.max(), point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#copy dataframe
cdata = pd.DataFrame.copy(data)
#convert temperature from Kelvin to Fahrenheit
cdata["Max Temp"] = round((cdata["Max Temp"]-273.15)*9/5+32,2)
#filter data
hotel_df = cdata[(cdata["Max Temp"] > 70) & (cdata["Max Temp"] < 80) & (cdata["Wind Speed"] < 10) & (cdata["Cloudiness"] == 0)]
#drop any missing data
hotel_df = hotel_df.dropna(how = 'any')
hotel_df = hotel_df.reset_index()
hotel_df

,index,lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,country,date
0,107,-16.48,-151.75,77.68,79,0,9.14,PF,1594463551
1,151,-20.32,118.57,77.00,16,0,3.60,AU,1594463560
2,169,57.51,50.50,74.07,50,0,2.91,RU,1594463564
3,172,20.93,-17.03,75.20,73,0,8.20,MR,1594463564
4,223,59.94,55.00,76.28,40,0,4.55,RU,1594463575
5,229,-18.77,46.03,79.65,33,0,3.84,MG,1594463576
6,277,51.15,41.23,78.42,44,0,4.35,RU,1594463585
7,356,39.36,-9.38,77.00,65,0,1.34,PT,1594463600
8,377,-14.25,129.55,78.12,57,0,3.11,AU,1594463604
9,448,27.98,-114.06,70.18,63,0,1.68,MX,1594463618


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#set up parameters to search
params = {
    "radius" : 5000,
    "keyword" : "hotel",  
    "key": g_key
}

hotel_df["Hotel name"] = ""
hotel_df["Hotel city"] = ""
hotel_df["Hotel country"] = ""
hotel_df["Hotel lat"] = ""
hotel_df["Hotel lng"] = ""

#loop through all rows in hotel df
for index, row in hotel_df.iterrows():
    lat = row["lat"]
    lng = row["lng"]
    params["location"] = f"{lat},{lng}"
    
    response = requests.get(url, params = params).json()
    results = response['results']
    
    #update hotel_df
    try:
        hotel_df.loc[index,"Hotel name"] = results[0]['name']
        
        #update city and country by splitting plus code
        temploc = results[0]["plus_code"]["compound_code"].split(' ', 1)
        temploc1 = temploc[1].split(',')
        hotel_df.loc[index,"Hotel city"] = temploc1[0]
        hotel_df.loc[index,"Hotel country"] = temploc1[1]
        
        #update lat and lng of hotel
        hotel_df.loc[index,"Hotel lat"] = results[0]["geometry"]["location"]["lat"]
        hotel_df.loc[index,"Hotel lng"] = results[0]["geometry"]["location"]["lng"]
        print(f'Processing Record #{index} , {hotel_df.loc[index,"Hotel name"]}')
    except:
        print(f"Cannot find the hotel for Record #{index}")
        


Processing Record #0 , Oa Oa Lodge
Processing Record #1 , ibis Styles Port Hedland
Cannot find the hotel for Record #2
Processing Record #3 , Free Zone Hotel Nouadhibou
Processing Record #4 , Gostinitsa V Kose
Processing Record #5 , Commissariat De Police Tsiroanomandidy
Cannot find the hotel for Record #6
Processing Record #7 , MH Peniche
Processing Record #8 , TDC Village (Hundred Man Camp)
Processing Record #9 , Hotel TerraSal


In [6]:
#clean hotel df data by dropping rows that doesn't contain hotel information
cleanhoteldf = pd.DataFrame.copy(hotel_df)
cleanhoteldf = cleanhoteldf[cleanhoteldf["Hotel city"] != ""]
cleanhoteldf = cleanhoteldf.reset_index(drop = True)
cleanhoteldf

,index,lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,country,date,Hotel name,Hotel city,Hotel country,Hotel lat,Hotel lng
0,107,-16.48,-151.75,77.68,79,0,9.14,PF,1594463551,Oa Oa Lodge,Vaitape,French Polynesia,-16.4989,-151.757
1,151,-20.32,118.57,77.00,16,0,3.60,AU,1594463560,ibis Styles Port Hedland,Port Hedland,Western Australia,-20.3087,118.61
2,172,20.93,-17.03,75.20,73,0,8.20,MR,1594463564,Free Zone Hotel Nouadhibou,Nouadhibou,Mauritania,20.939,-17.0345
3,223,59.94,55.00,76.28,40,0,4.55,RU,1594463575,Gostinitsa V Kose,Kordon,Perm Krai,59.953,55.0478
4,229,-18.77,46.03,79.65,33,0,3.84,MG,1594463576,Commissariat De Police Tsiroanomandidy,Tsiroanomandidy,Madagascar,-18.7679,46.0516
5,356,39.36,-9.38,77.00,65,0,1.34,PT,1594463600,MH Peniche,Peniche,Portugal,39.357,-9.36314
6,377,-14.25,129.55,78.12,57,0,3.11,AU,1594463604,TDC Village (Hundred Man Camp),Wadeye,Northern Territory,-14.2376,129.519
7,448,27.98,-114.06,70.18,63,0,1.68,MX,1594463618,Hotel Los Caracoles,Guerrero Negro,Baja California Sur,27.9678,-114.028


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel name}</dd>
<dt>City</dt><dd>{Hotel city}</dd>
<dt>Country</dt><dd>{Hotel country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in cleanhoteldf.iterrows()]
locations = cleanhoteldf[["Hotel lat", "Hotel lng"]]

In [31]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))